In [46]:
import torch
import torch.nn as nn
import torch.optim as opti
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import random_split
from tqdm import tqdm

In [47]:
data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

In [48]:
train_size = int(0.8 * len(data))
val_size = len(data) - train_size

train_dataset, test_dataset = random_split(data, [train_size, val_size])

In [49]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [50]:
x,y  = train_dataset[0]

In [51]:
x.shape

torch.Size([1, 28, 28])

In [52]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [53]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = opti.SGD(model.parameters(), lr=0.001)


num_epochs = 8
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    train_dataloader = tqdm(train_dataloader)
    for images, labels in train_dataloader:
        optimizer.zero_grad() 
        outputs = model(images) 
        loss = criterion(outputs, labels)  
        loss.backward() 
        optimizer.step() 

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_dataloader):.4f}")

100%|██████████| 1500/1500 [00:16<00:00, 89.99it/s]


Epoch [1/8], Loss: 2.1481


100%|██████████| 1500/1500 [00:16<00:00, 91.06it/s]


Epoch [2/8], Loss: 1.1672


100%|██████████| 1500/1500 [00:16<00:00, 90.93it/s]


...

Epoch [6/8], Loss: 0.6217


100%|██████████| 1500/1500 [00:17<00:00, 87.01it/s]


Epoch [7/8], Loss: 0.5982


100%|██████████| 1500/1500 [00:17<00:00, 86.76it/s]

Epoch [8/8], Loss: 0.5795


In [54]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    test_dataloader = tqdm(test_dataloader)
    for images, labels in test_dataloader:

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total:.2f}%")


100%|██████████| 375/375 [00:01<00:00, 199.82it/s]

Validation Accuracy: 80.33%
